In [147]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os, sys, random,datetime, math
import requests 
import shutil 
import json
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import keras

In [169]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, optimizers,losses,regularizers
from tensorflow.keras.layers import InputLayer, Dense, experimental, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,LeakyReLU,GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Rescaling, Softmax
from tensorflow.keras.applications import VGG16
from keras.preprocessing.image import load_img,img_to_array,array_to_img
from tensorflow.keras.models import Model, load_model

In [172]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_dataset = image_generator.flow_from_directory(batch_size=16,
                                                 directory='./imgs',
                                                 shuffle=True,
                                                 target_size=(256, 256), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=16,
                                                 directory='./imgs',
                                                 shuffle=True,
                                                 target_size=(256, 256), 
                                                 subset="validation",
                                                 class_mode='categorical')

Found 3483 images belonging to 12 classes.
Found 865 images belonging to 12 classes.


In [44]:
# function for download test image
def download_img_url(image_url):
    filename = './test_imgs/'+image_url.split("/")[-1]
    r = requests.get(image_url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived')

In [138]:
model.compile(loss = losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
              optimizer='rmsprop',
              metrics=['accuracy'])

In [139]:
nb_train_samples = len(train_dataset.filenames)
nb_validation_samples = len(validation_dataset.filenames)
epochs = 8
batch_size = 16

In [103]:
train_dataset.class_indices

{'BAG': 0,
 'DRESS': 1,
 'JEWELRY': 2,
 'JUMPSUIT': 3,
 'LINGERIE': 4,
 'OTHERS': 5,
 'OUTERWEAR': 6,
 'PANT': 7,
 'SHOE': 8,
 'SHORTS': 9,
 'SKIRT': 10,
 'TOPS': 11}

In [180]:
# bulid network
inputs = Input(shape=[256, 256, 3])
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
for l in base_model.layers:
    l.trainable = False # 第一步锁定前层，不进行训练
fla = Flatten()(base_model.output)
fc6 = Dense(2048, activation='relu')(fla) # 新增加的待训练全连接层fc6
drop6 = Dropout(rate=0.5)(fc6)
fc7 = Dense(512, activation='relu')(drop6) # 新增加的待训练全连接层fc7
drop7 = Dropout(rate=0.5)(fc7)
fc8 = Dense(12, activation='softmax')(drop7) # 新增加的待训练全连接层fc8，节点数等于分类类别数
model = Model(inputs=inputs, outputs=fc8) # 完成Model构建

# compile model
optimizer = Adam(lr=0.00001)
model.summary() # 可打印模型概况
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# train
model.fit_generator(
    train_dataset,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_dataset,
    validation_steps=nb_validation_samples // batch_size)
# save model to h5 file
model.save(filepath='./fv_vgg16_4.h5')


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0  

In [193]:
from tensorflow.keras.preprocessing import image

def classify(img_path, model):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_batch /= 255.
    prediction = model.predict(img_batch)
    return prediction

In [212]:
prediction = {}
for dirname in os.listdir("./test_imgs/"):
    result = list(classify("./test_imgs/"+dirname,model)[0])
    cls = result.index(max(result))
    prediction[dirname]=cls
    

In [200]:
train_dataset.class_indices

{'BAG': 0,
 'DRESS': 1,
 'JEWELRY': 2,
 'JUMPSUIT': 3,
 'LINGERIE': 4,
 'OTHERS': 5,
 'OUTERWEAR': 6,
 'PANT': 7,
 'SHOE': 8,
 'SHORTS': 9,
 'SKIRT': 10,
 'TOPS': 11}

In [202]:
prediction

{'f4674c7ddb930140f28d7de9ebc23f3bd0119605.jpg': 4,
 'cfc8fc03ee0c7edda7fbb0a94aeafa80fdf15fdd.jpg': 2,
 '49d0a92ac77f3632f1de09fbfda6f29ede23355d.jpg': 11,
 'c76dc2d34f2f6457af14279b697790caeb52651b.jpg': 0,
 'c63e1cb7471f65f90d3e0265759c9b9c209276d1.jpg': 11,
 'f0dfbbec1c708be4c004061e8e73c303df3e2a9a.jpg': 11,
 '657fabe8d6599af3d332063fec443fcc7a3aac6a.jpg': 2,
 '8fe724aa080f83dd7fe157665525b40c5c2ce5d4.jpg': 6,
 '6589378e70608ee67c3588933e7c39d32956b037.jpg': 11,
 '5ab3910ba86c6451327d23543265ce7790e36178.jpg': 7,
 '8e119273f2ad78d3b21b4ce75664fdc7d0259f35.jpg': 11,
 '122f236ce290def1b888517ea6f4480b1635cd5d.jpg': 11,
 'e560bbd3605e0a975e3b1aeab6cd3ddb73d08923.jpg': 8,
 'e2f680e996a4d89d89ea42a3ab30ecbb3d23add5.jpg': 11,
 'b489814a650f8c38259de16b7057750692e604ea.jpg': 11,
 '7a1cde968f8af9c6de349c81d509bede577df606.jpg': 5,
 '736a68cefe5f5a0e087bd5b2dea8e11df61032bc.jpg': 8,
 'c17c1f6b8edc8264406832e31ca7f8e87b4431b7.jpg': 5,
 'da8ee94157ced4abd796fe3c838ea28d97b3371b.jpg': 3,
 'c1

In [207]:
class_map ={}
for k,v in train_dataset.class_indices.items():
    class_map[v]=k

In [208]:
class_map

{0: 'BAG',
 1: 'DRESS',
 2: 'JEWELRY',
 3: 'JUMPSUIT',
 4: 'LINGERIE',
 5: 'OTHERS',
 6: 'OUTERWEAR',
 7: 'PANT',
 8: 'SHOE',
 9: 'SHORTS',
 10: 'SKIRT',
 11: 'TOPS'}

In [225]:
test_result = []
with open('product_data.json') as f:
    data = json.loads(f.read())
for d in data:
    image_name= d['image_url'].split('/')[-1]
    predict_num = prediction[image_name]
    predict_cls = class_map[predict_num]
    d['category'] = predict_cls

In [231]:
with open('product_data.json', 'w') as fout:
    json.dump(data , fout)